In [1]:
import requests
import pandas as pd
import numpy as np
import math
from bs4 import BeautifulSoup
from datetime import datetime

In [2]:
url= 'https://www2.hm.com/en_us/men/products/jeans.html'

headers = {'User-Agent':'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_5) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/50.0.2661.102 Safari/537.36 OPR/37.0.2178.54'}

page = requests.get( url, headers=headers )

soup = BeautifulSoup(page.text, 'html.parser')

products = soup.find( 'ul', class_='products-listing small' )

products_list= products.find_all ('article',class_="hm-product-item")

#product id

products_id = [p.get('data-articlecode') for p in products_list]

#product category

products_category= [p.get('data-category') for p in products_list]

#product name 

#extraction all the name 
products_list_2=  products.find_all('a', class_='link')

products_name =[p.get_text('link') for p in products_list_2]

#product price

#extraction informations of price
products_list_3 = products.find_all('span', class_='price regular')

#find all the price in page
products_price= [p.get_text('price_regular').replace('$ ','') for p in products_list_3]

data = pd.DataFrame([products_id,products_category,products_name,products_price]).T

data.columns = ['products_id','products_category','products_name','products_price']

data['scrapy_datetime'] = datetime.now().strftime('%Y-%m-%d-%H-%M:%S')

In [10]:
#key browser
headers = {'User-Agent':'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_5) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/50.0.2661.102 Safari/537.36 OPR/37.0.2178.54'}

#empty Data Frame (to insert the joins)
df_details= pd.DataFrame()

#API Requests
for i in range(len(data)):
    
    url= 'https://www2.hm.com/en_us/productpage.'+ data.loc[i,'products_id'] +'.html'

    
    page = requests.get( url, headers=headers )


    #=======multiples products =====================#

    #Beautifull Souo object
    soup= BeautifulSoup(page.text,'html.parser')

    ### ===================== color name ======================

    products_list= soup.find_all('a',class_='filter-option miniature')

    #product id
    products_id = [p.get('data-articlecode') for p in products_list]

    #color name
    color_name = [p.get('data-color') for p in products_list]


    #create data frame

    df_color = pd.DataFrame([products_id,color_name]).T

    #rename the columns of DF
    df_color.columns= ['product_id','color_name']

    #create new colum styly_id
    df_color['stlye_id'] = df_color['product_id'].apply(lambda x: x[:-3])

    #create new colum color_id
    df_color['color_id'] = df_color['product_id'].apply(lambda x: x[-3:])

    # ============================== composition name =============================== #

    #composition
    composition_list= soup.find_all('div', class_ = 'details-attributes-list-item')

    product_composition = [list(filter (None, p.get_text().split('\n') ) ) for p in composition_list]

    #created the dataframe aux
    df_aux= pd.DataFrame(product_composition).T
    #renome the columns
    df_aux.columns= df_aux.iloc[0]

    #select columns 
    df_aux= df_aux[['Fit','Composition','Art. No.']]

    #delete the first row and delete the columns that is the empty
    df_aux= df_aux.iloc[1:3]

    #copying the code and the model to row the below
    df_aux = df_aux.fillna(method='ffill')

    #rename the data frame to a new variable
    df_composition = df_aux.copy(True)

    #create new colum styly_id
    df_composition['stlye_id'] =df_composition['Art. No.'].apply(lambda x: x[:-3])

    #create new colum color_id
    df_composition['color_id'] = df_composition['Art. No.'].apply(lambda x: x[-3:])

    #merge dataframe of color and composition 
    data_sku= pd.merge(df_color,df_composition[['stlye_id','Fit','Composition']], how='left', on ='stlye_id')
    
    df_details= pd.concat([df_details,data_sku],axis=0)

In [14]:
df_details

,product_id,color_name,stlye_id,color_id,Fit,Composition
0,0985159002,Denim blue,0985159,002,Skinny fit,"Shell: Cotton 99%, Spandex 1%"
1,0985159002,Denim blue,0985159,002,Skinny fit,Pocket lining: Cotton 100%
2,0985159003,Dark gray,0985159,003,Skinny fit,"Shell: Cotton 99%, Spandex 1%"
3,0985159003,Dark gray,0985159,003,Skinny fit,Pocket lining: Cotton 100%
4,0985159004,Light denim blue,0985159,004,Skinny fit,"Shell: Cotton 99%, Spandex 1%"
...,...,...,...,...,...,...
11,0938875012,Denim blue,0938875,012,Slim fit,Pocket lining: Cotton 100%
12,0938875013,Pale denim blue,0938875,013,Slim fit,"Cotton 99%, Spandex 1%"
13,0938875013,Pale denim blue,0938875,013,Slim fit,Pocket lining: Cotton 100%
14,0938875014,Black,0938875,014,Slim fit,"Cotton 99%, Spandex 1%"
